In [1]:
# Imports related to data processing.
import json
import gzip
from os import path
# TODO: add instructions for installing nltk: 
# - pip3 install -r requirements.txt (used pip3v20)
# - sudo python -m nltk.downloader -d /usr/local/share/nltk_data all
# TEMP | see: https://www.nltk.org/install.html and https://www.nltk.org/data.html
# TEMP | API: https://www.nltk.org/api/nltk.corpus.html
from nltk.corpus import brown

In [19]:
# Define some constant strings pointing to the UCI Amazon review data.
DATA_DIR = '../data'
UCI_DATA_DILE = 'uci_labelled_data/amazon_cells_labelled.txt'

# Read in sentiment laballed sentences UCI's Amazon data file.
uci_sentiment_data = []
with open(path.join(DATA_DIR, UCI_DATA_DILE), encoding='utf-8') as fileObj:
    uci_sentiment_data = [line.rstrip('\n') for line in fileObj]
    
# Split each line into a tuple containing the text in the 0th index and the sentiment label in the 1st index.
for index in range(0, len(uci_sentiment_data)):
    split_sentence = uci_sentiment_data[index].split('\t')
    uci_sentiment_data[index] = (split_sentence[0], split_sentence[1])
    
print('Read-in UCI sentiment-labelled Amazon data, count: {}.'.format(len(uci_sentiment_data)))

Processed UCI sentiment-labelled Amazon data, count: 1000.


In [20]:
# Define some constant strings pointing to the UCSD Amazon review data.
UCSD_DATA_DIR = path.join('../data', 'ucsd_amazon_data')
UCSD_DATA_BEAUTY = 'All_Beauty_5.json.gz'
UCSD_DATA_FASHION = 'AMAZON_FASHION_5.json.gz'
UCSD_DATA_APPLIANCES = 'Appliances_5.json.gz'

# Read in JSON data from UCSD's Amazon data files -- Beauty data.
ucsd_beauty_data = []
with gzip.open(path.join(UCSD_DATA_DIR, UCSD_DATA_BEAUTY)) as json_gzip:
    for line in json_gzip:
        ucsd_beauty_data.append(json.loads(line))

# -- Beauty data.
ucsd_fashion_data = []
with gzip.open(path.join(UCSD_DATA_DIR, UCSD_DATA_FASHION)) as json_gzip:
    for line in json_gzip:
        ucsd_fashion_data.append(json.loads(line))

# -- Appliance data.
ucsd_appliances_data = []
with gzip.open(path.join(UCSD_DATA_DIR, UCSD_DATA_APPLIANCES)) as json_gzip:
    for line in json_gzip:
        ucsd_appliances_data.append(json.loads(line))
        
print('Read-in UCSD Amazon data. Beauty count: {}, Fashion count: {}, Appliances count: {}.'.format(len(ucsd_beauty_data), len(ucsd_fashion_data), len(ucsd_appliances_data)))

Processed UCSD Amazon data. Beauty count: 5269, Fashion count: 3176, Appliances count: 2277.


In [ ]:
# TEMP Brown Experiments.
#print(", ".join(brown.words()))